| Experiment Name | Description | Parameters |
| :- | :- | :- |
| EXP11 |   Reference run with no forcings at rest  | rnDt              = 900.                  |
| EXP12 |   EXP11 without flattening of the equator | ln_equ_flat       = false                 |
| EXP13 |   Geostrophic adjustment                  | nn_init           = 5                     | --> crash |
| EXP14 |   Week winds, no other forcing.           | rn_trp, rn_srp    = 0, ln_qsr = false     |

In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import cftime._cftime as cft
#import holoviews as hv

#hv.extension('bokeh')

In [ ]:
def clean_coords(xrdataset, start_year):
    try:
        shift = cft.timedelta(days=360 * start_year)
        datetimeindex = (xrdataset.indexes['time_counter'] + shift).to_datetimeindex
    except:
        datetimeindex = xrdataset.indexes['time_counter']
    try:
        latitude  = xrdataset.nav_lat[:,0].values
        longitude = xrdataset.nav_lon[0,:].values
    except:
        latitude  = xrdataset.nav_lat_grid_T[:,0].values
        longitude = xrdataset.nav_lon_grid_T[0,:].values
    try:
        xrdataset = xrdataset.rename_dims({'x_grid_T': 'x' ,'y_grid_T': 'y'})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3t.cumsum('deptht')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3u.cumsum('depthu')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3v.cumsum('depthv')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3w.cumsum('depthw')})
    except:
        pass
    return xrdataset.assign_coords(dict({'time_counter':datetimeindex, 'y': latitude, 'x': longitude}))

In [ ]:
def read_abort(path, dict_vars={'votemper': 'T', 'vosaline': 'S', 'vozocrtx': 'U', 'vomecrty': 'V', 'vovecrtz': 'W', 'sossheig': 'H', 'vovvldep': 'depth'}):
    abort = xr.load_dataset(path)
    abort = abort.rename(dict_vars)
    abort = abort.set_coords(['nav_lon', 'nav_lat', 'depth'])
    abort = abort.assign_coords({'nav_lev': abort.nav_lev * -1})
    abort = abort.assign_coords({'depth': abort.depth * -1})
    return(abort[list(dict_vars.values())])


# **EXP11: Reference Run**

In [ ]:
U1       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP11/BASIN_grid_U.nc")
T1       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP11/BASIN_grid_T.nc")
mask1    = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP11/mesh_mask.nc")

In [ ]:
T1 = clean_coords(T1, 2000)
U1 = clean_coords(U1, 2000)
mask1 = clean_coords(mask1, 2000)

In [ ]:
T1.soce.mean(['deptht', 'y', 'x']).plot()

## Bathymetry

In [ ]:
mask1.e3t_0.sum(dim='nav_lev').plot(x='x', y='y')

## Salinity along the center of the BASIN

In [ ]:
%%output holomap='scrubber'
%%opts QuadMesh style(cmap='RdBu_r') plot[colorbar=True] 
%%opts QuadMesh [width=500, height=400]
hv_ds = hv.Dataset(T1.soce.isel(x=20))
hv_ds.to(hv.QuadMesh, ['y', 'deptht'])

## Zonal velocity along the center of the BASIN

In [ ]:
%%output holomap='scrubber'
%%opts QuadMesh style(cmap='RdBu_r') plot[colorbar=True] 
%%opts QuadMesh [width=500, height=400]
hv_ds = hv.Dataset(U1.uoce.isel(x=20))
hv_ds.to(hv.QuadMesh, ['y', 'depthu'])


# **EXP12: Reference Run without flatten topography at the equator**

In [ ]:
U2       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP12/BASIN_grid_U.nc")
T2       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP12/BASIN_grid_T.nc")
mask2    = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP12/mesh_mask.nc")

In [ ]:
T2 = clean_coords(T2, 2000)
U2 = clean_coords(U2, 2000)
mask2 = clean_coords(mask2, 2000)

## Bathymetry

In [ ]:
mask2.e3t_0.sum(dim='nav_lev').plot(x='x', y='y')

## Salinity along the center of the BASIN

In [ ]:
T2.soce.isel(x=21, time_counter=-1).plot(y='z')

## Zonal velocity along the center of the BASIN

In [ ]:
U2.uoce.isel(x=20, time_counter=-1).plot(y='z')
plt.title('')


# **EXP13: Geostrophic Adjustment**

In [ ]:
U3       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP13/BASIN_grid_U.nc")
T3       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP13/BASIN_grid_T.nc")
#mask3       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP13/mesh_mask.nc")

In [ ]:
T3 = clean_coords(T3, 2000)
U3 = clean_coords(U3, 2000)

In [ ]:
%%output holomap='scrubber'
%%opts QuadMesh style(cmap='RdBu_r') plot[colorbar=True] 
%%opts QuadMesh [width=500, height=400]
hv_ds = hv.Dataset(T3.ssh)
hv_ds.to(hv.QuadMesh, ['x', 'y'])


# **EXP17: Standard Iso-Neutral Tracer Diffusion**

In [ ]:
U7       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP17/BASIN_grid_U.nc")
T7       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP17/BASIN_grid_T.nc")

T7 = clean_coords(T7, 2000)
U7 = clean_coords(U7, 2000)

#abort4 = read_abort('/home/dkamm/NEMOGCM/tests/BASIN2/EXP14/output.abort.nc')

In [ ]:
#T5 = T5.assign_coords({'z': 5. -mask5.e3t_0.cumsum('nav_lev').rename({'nav_lev': 'deptht'}).isel(time_counter=0)})

In [ ]:
%%output holomap='scrubber'
%%opts QuadMesh style(cmap='RdBu_r') plot[colorbar=True] 
%%opts QuadMesh [width=500, height=400]
hv_ds = hv.Dataset(U7.uoce.isel(depthu=0))
hv_ds.to(hv.QuadMesh, ['x', 'y'])

In [ ]:
(5. -mask5.e3t_0.cumsum('nav_lev').rename({'nav_lev': 'deptht'}).isel(time_counter=0, y=10)).plot()

In [ ]:
U7.utau.isel(time_counter=-1).plot()

In [ ]:
U7.uoce

In [ ]:
U7       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP_dom7/BASIN_grid_U.nc")
T7       = xr.load_dataset("/home/dkamm/NEMOGCM/tests/BASIN2/EXP_dom7/BASIN_grid_T.nc")

T7 = clean_coords(T7, 2000)
U7 = clean_coords(U7, 2000)

In [ ]:
U7

In [ ]:
U7_0       = xr.load_dataset("/home/dkamm/NEMO4.0/tests/BASIN_compiled_v0.1.0/EXP_dom/BASIN_grid_U.nc")
T7_0       = xr.load_dataset("/home/dkamm/NEMO4.0/tests/BASIN_compiled_v0.1.0/EXP_dom/BASIN_grid_T.nc")

T7_0 = clean_coords(T7_0, 2000)
U7_0 = clean_coords(U7_0, 2000)

In [ ]:
diff = dom_42 - dom_40

In [ ]:
diff.gdepw_0.isel(nav_lev=-1, time_counter=0).plot()